# Préparation du jeu de données PubChem - Ajout du bruit (Tesla)

Dans ce notebook, on va définir les fonctions créant les fichiers de données qui seront utilisées pour l'entraînement et la validation du modèle (coordonnées bruitées et labels de différences de distances attendues)

On définit pour cela une fonction prenant les chemins du jeu à préparer et le chemin de deux fichiers h5 à créer : le jeu contenant les entrées du RN pour tous les exemples (un tableau de 1000 floats par molécule contenant les distances des atomes au repère et leurs masses), et le jeu contenant les cibles (un tableau de 800 floats par molécule contenant les delta distances à calculer en mÅ)

Cette solution de créer des fichiers de données préparées plutôt que de préparer les données en mémoire durant l'apprentissage par le RN va faciliter l'écriture du code d'entraînement du RN et permet de tester plus facilement les différentes étapes indépendantes de l'entraînement du modèle, mais elle possède l'inconvénient de créér des fichiers très volumineux. La base de données actuelle étant de taille raisonnable et 3To de disque étant disponibles, les limites ne sont pas atteintes actuellement, mais il faudrait passer à une solution de préparation des données dynamique lors de l'apprentissage si l'on souhaite entraîner un modèle à partir de milliards d'exemples par la suite. 

#### Définition des chemins de fichiers

In [7]:
test_set_location = "../data/test_set_riken_v2.h5"
test_set_prepared_input_location = "../data/test_set_riken_v2_prepared_input.h2"
test_set_labels_location = "../data/test_set_riken_v2_labels.h2"

train_set_location = "../data/train_set_riken_v2.h5"
train_set_prepared_input_location = "../data/train_set_riken_v2_prepared_input.h5"
train_set_labels_location = "../data/train_set_riken_v2_labels.h5"

minimal_set_riken_location = "../data/minimal_set_riken_v2.h5"
minimal_set_prepared_input_location = "../data/minimal_set_riken_v2_prepared_input.h5"
minimal_set_labels_location = "../data/minimal_set_riken_v2_labels.h5"

#### Définition de la fonction d'ajout de bruit

In [8]:
import numpy as np

def positions_bruitees(positions):    
    bruit = np.random.normal(loc=0.0, scale=0.028867, size=positions.shape)
    return ((positions + bruit), bruit)


#### Définition de la fonction de calcul de la matrice de distances compressée à partir de la matrice des coordonnées des atomes

In [9]:
def matrice_distances_compr(positions):
    """ Renvoie la matrice de distances compressée des positions des atomes passées en paramètres
    La matrice de distances compressée est définie de la façon suivante : pour chaque atome, on calcule
    la distance avec chaque point du repère. Une ligne i de la matrice (n,4) correspond aux distances
    de l'atome i avec chacun des quatre points du repère"""
    
    nb_at = len(positions)
    
    # On renvoie un tableau vide si la molécule est vide
    if nb_at == 0:
        return []
    
    repere = np.array([[0, 0, 0], [1, 0, 0], [0, 1, 0], [0, 0, 1]])
    repere = np.vstack([repere]*nb_at)

    positions = np.tile(positions, 4).reshape(4*nb_at, 3)
    
    return np.sqrt(np.sum(np.power(positions-repere, 2), 1)).reshape(nb_at, 4)


#### Définition de la fonction de création des fichiers h5 de données et de labels

In [10]:
import h5py
import time
import numpy as np

def creation_input_RN(set_location, input_rn_location, labels_location):
    
    start_time = time.time()
    
    mol_vides = 0
    
    print("Creating input and label sets for "+set_location+" : ")
    
    print("Loading data...")
    # On charge le jeu de données original (en lecture seule)
    original_dataset_h5 = h5py.File(set_location, 'r')
    
    # On enregistre la taille du jeu de données
    taille = len(original_dataset_h5["anums"])
        
    # On créé les jeu deux données d'entrée du RN et de labels
    input_rn_dataset_h5 = h5py.File(input_rn_location, 'w')
    labels_dataset_h5 = h5py.File(labels_location, 'w')
    
    
    try:
    
        # Définition du type pour les tableaux de floats
        varlen_floatarray = h5py.special_dtype(vlen=np.dtype("float32"))

        print("Creating new files...")
        # On créé les datasets input et target
        input_dataset = input_rn_dataset_h5.create_dataset("inputs", shape=(taille,),
                                           dtype=varlen_floatarray, compression="gzip", 
                                           chunks=True, maxshape=(None,))

        targets_dataset = labels_dataset_h5.create_dataset("targets", shape=(taille,),
                                           dtype=varlen_floatarray, compression="gzip", 
                                           chunks=True, maxshape=(None,))


        print("Computing input and label sets...")
        # On parcourt toutes les molécules de l'exemple
        for i in range(taille):

            if i%10000 == 0:
                print("Computing input and label sets for molecule "+str(i)+" ("+str(i/taille*100)+"%)")

            # On récupère les coordonnées de la molécule courante et on y ajoute du bruit
            coords = np.array(original_dataset_h5["riken_coords"][i]).reshape(-1,3)
            dist_init = matrice_distances_compr(coords)
            coords_bruit, bruit = positions_bruitees(coords)
            coords_bruit = coords_bruit.reshape(-1, 3)
            bruit = bruit.reshape(-1, 3)
            
            if len(coords) == 0:
                print("Molécule vide")
                mol_vides += 1
            else:
                # On calcule les différence de distances cibles (en mÅ) et les distances bruitées (en Å)
                dist_bruit = matrice_distances_compr(coords_bruit)
                delta_dist_targets = (dist_init - dist_bruit)*1000

            # On récupère les masses atomiques de la molécule courante
            masses = original_dataset_h5["amasses"][i]

            # On initialise l'entrée du RN et le vecteur cible pour la molécule courante
            entree_courante = np.zeros(shape=(1000, 1))
            cible_courante = np.zeros(shape=(200, 4))

            # On ajoute les coordonnées bruitées et les masses à l'entrée avec padding, et les coordonnées
            # cibles au dataset targets
            j=0
            for masse in masses:
                
                # Ajout des données au vecteur entrée
                index_input_courant = j*5
                entree_courante[index_input_courant] = dist_bruit[j][0]
                entree_courante[index_input_courant+1] = dist_bruit[j][1]
                entree_courante[index_input_courant+2] = dist_bruit[j][2]
                entree_courante[index_input_courant+3] = dist_bruit[j][3]
                entree_courante[index_input_courant+4] = masse
                
                # Ajout des données à la matrice cibles
                cible_courante[j] = delta_dist_targets[j]

                j+=1

            # On aplatit le vecteur cibles
            cible_courante = cible_courante.reshape(1, 800)
            
            # On insère les données dans le fichier h5 en mémoire
            input_dataset[i] = entree_courante.reshape(-1, 1000)
            targets_dataset[i] = cible_courante

        print("Writing datasets on disk...")
        # On écrit les datasets sur le disque
        input_rn_dataset_h5.flush()
        labels_dataset_h5.flush()
        
        print(str(mol_vides)+" molécules vides au total")


        print(input_rn_location+" and "+labels_location+" have been correctly written on disk")
        print("--- %s seconds ---" % (time.time() - start_time))
        
    finally:
        original_dataset_h5.close()
        input_rn_dataset_h5.close()
        labels_dataset_h5.close()




In [11]:
total_start_time = time.time()

#### Préparation des données d'entrée du RN et des labels pour le jeu minimal

In [12]:
print()
print("Préparation données jeu minimal :")
creation_input_RN(minimal_set_riken_location, minimal_set_prepared_input_location,
                  minimal_set_labels_location)


Préparation données jeu minimal :
Creating input and label sets for ../data/minimal_set_riken_v2.h5 : 
Loading data...
Creating new files...
Computing input and label sets...
Computing input and label sets for molecule 0 (0.0%)
Computing input and label sets for molecule 10000 (10.0%)
Computing input and label sets for molecule 20000 (20.0%)
Computing input and label sets for molecule 30000 (30.0%)
Computing input and label sets for molecule 40000 (40.0%)
Computing input and label sets for molecule 50000 (50.0%)
Computing input and label sets for molecule 60000 (60.0%)
Computing input and label sets for molecule 70000 (70.0%)
Computing input and label sets for molecule 80000 (80.0%)
Computing input and label sets for molecule 90000 (90.0%)
Writing datasets on disk...
0 molécules vides au total
../data/minimal_set_riken_v2_prepared_input.h5 and ../data/minimal_set_riken_v2_labels.h5 have been correctly written on disk
--- 138.18809986114502 seconds ---


#### Préparation des données d'entrée du RN et des labels pour le jeu d'entraînement

In [ ]:
print()
print("Préparation données jeu d'entraînement :")
creation_input_RN(train_set_location, train_set_prepared_input_location, train_set_labels_location)

#### Préparation des données d'entrée du RN et des labels pour le jeu de validation

In [ ]:
print()
print("Préparation données jeu de validation :")
creation_input_RN(test_set_location, test_set_prepared_input_location, test_set_labels_location)

In [13]:
print("Total time : ")
print("--- %s seconds ---" % (time.time() - total_start_time))

Total time : 
--- 138.90174651145935 seconds ---
